In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit, learning_curve
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.preprocessing import Imputer, MinMaxScaler, StandardScaler

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

import zipfile

from IPython.display import Image

In [ ]:
path = '/kaggle/input/home-credit-default-risk/'

POS_CASH_balance = pd.read_csv(path+'POS_CASH_balance.csv')
bureau_balance = pd.read_csv(path+'bureau_balance.csv')
application_train = pd.read_csv(path+'application_train.csv')
previous_application = pd.read_csv(path+'previous_application.csv')
installments_payments = pd.read_csv(path+'installments_payments.csv')
credit_card_balance = pd.read_csv(path+'credit_card_balance.csv')
application_test = pd.read_csv(path+'application_test.csv')
bureau = pd.read_csv(path+'bureau.csv')

In [ ]:
# Выведем изображение с моделью данных
Image(url = "https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png")

In [ ]:
# Выведем shape'ы тренировочных и тестовых данных
print('application_train shape: {} rows, {} columns'.format(*application_train.shape))
print('application_test shape: {} rows, {} columns'.format(*application_test.shape))

In [ ]:
application_train.set_index('SK_ID_CURR', inplace=True)
application_test.set_index('SK_ID_CURR', inplace=True)

y = application_train['TARGET']

In [ ]:
# Определим категориальные и вещественные признаки
categorical_features = [col for col in application_test.columns if application_test[col].dtype == 'object']
numerical_features = [col for col in application_test.columns if application_test[col].dtype != 'object']
        
print('Data has {} categorical features, and {} numerical features'.format(
    len(categorical_features), len(numerical_features)))

In [ ]:
# Запилим функцию для визуализации распределения вещественных признаков
def plot_features_hist(df, features, cols=3, bins=200, window_width=7.5, window_height=5):
    cols = cols
    rows = (len(features) + cols - 1) // cols
    gs = gridspec.GridSpec(rows, cols)
    fig = plt.figure(figsize=(cols * window_width, rows * window_height))
    for feature, grd in zip(features,
                            range(len(features))):
        ax = plt.subplot(gs[grd // cols, grd % cols])
        fig = plt.hist(df[feature].dropna(), bins=bins)
        plt.title(str(feature)
                  +' (min:'+str(round(min(df[feature].dropna())))
                  +', mean:'+str(round(np.mean(df[feature].dropna())))
                  +', max:'+str(round(max(df[feature].dropna())))+')')
    plt.show()

In [ ]:
plot_features_hist(application_train, numerical_features)

In [ ]:
plot_features_hist(application_test, numerical_features)

In [ ]:
print('application_test "DAYS_EMPLOYED" anomalies {}, {}%'.format(
    len(application_test[application_test['DAYS_EMPLOYED']==365243]),
    len(application_test[application_test['DAYS_EMPLOYED']==365243]) / len(application_test) * 100))
print('')
print('application_train "DAYS_EMPLOYED" anomalies {}, {}%'.format(
    len(application_train[application_train['DAYS_EMPLOYED']==365243]),
    len(application_train[application_train['DAYS_EMPLOYED']==365243]) / len(application_train) * 100))

В признаке DAYS_EMPLOYED часто встречается значение 365243. Скорее всего, это какая-то опечатка или характерный баг CRM-ки, не может же сотрудник работать тысячу лет на одном месте. Создадим специальный признак, в котором будет метка о наличии такого явления, а само значение в признаке заменим пропуском.

In [ ]:
application_train['DAYS_EMPLOYED_ANOM'] = application_train["DAYS_EMPLOYED"] == 365243
application_train["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)
application_test['DAYS_EMPLOYED_ANOM'] = application_test["DAYS_EMPLOYED"] == 365243
application_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

Далее обработаем категориальные признаки

In [ ]:
application_train = pd.get_dummies(data=application_train, columns=categorical_features, dummy_na=True)
application_test = pd.get_dummies(data=application_test, columns=categorical_features, dummy_na=True)

In [ ]:
print('application_train shape: {} rows {} columns'.format(*application_train.shape))
print('application_test shape: {} rows {} columns'.format(*application_test.shape))

Тренировочный и тестовый наборы теперь имеют разное число признаков. Приведём их к единому виду

In [ ]:
application_train, application_test = application_train.align(application_test, join='inner', axis = 1)

print('application_train shape: ', application_train.shape)
print('application_test shape: ', application_test.shape)

Далее разберёмся с пропусками

In [ ]:
missing_df = (application_train.isna().sum() / len(application_train)).reset_index()
missing_df.sort_values(ascending=False, by=0)

In [ ]:
def missing_indicator(df, features=None, inplace=False):
    if not features:
        features = df.columns
    if not inplace:
        df = df.copy()
    for feature in df[features].columns:
        if df[feature].isna().sum() > 0:
            df['missing_'+feature] = df[feature].isna().astype(int)
    return df

In [ ]:
application_train = missing_indicator(application_train)
application_test = missing_indicator(application_test)

print('application_train shape: ', application_train.shape)
print('application_test shape: ', application_test.shape)

По всей видимости, в тренировочном и тестовом наборах разный характер пропусков, приведём к единому виду

In [ ]:
application_train, application_test = application_train.align(application_test, join='inner', axis = 1)

print('application_train shape: ', application_train.shape)
print('application_test shape: ', application_test.shape)

Далее мы заполним пропуски в вещественных признаках средними значениями. Однако среди вещественных принаков встречаются бинарные признаки, и пропуски в них мы будем заполнять строго нулями

In [ ]:
binary_features_train = application_train[numerical_features].nunique()
binary_features_train = binary_features_train[binary_features_train<=2]
binary_features_train = binary_features_train.index

binary_features_test = application_test[numerical_features].nunique()
binary_features_test = binary_features_test[binary_features_test<=2]
binary_features_test = binary_features_test.index

min(binary_features_train == binary_features_test)

In [ ]:
binary_features_train

In [ ]:
application_train[binary_features_train] = application_train[binary_features_train].fillna(0)
application_test[binary_features_test] = application_test[binary_features_test].fillna(0)

In [ ]:
mean_imputer = Imputer(missing_values='NaN', strategy='mean')
application_train[numerical_features] = mean_imputer.fit_transform(application_train[numerical_features])
application_test[numerical_features] = mean_imputer.transform(application_test[numerical_features])

Теперь проверим, нет ли у нас признаков скореллированных с целевой переменной

In [ ]:
application_train.corrwith(y).sort_values(ascending=False)

In [ ]:
application_train.corrwith(y).sort_values()

### Model selection

In [ ]:
!rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM

In [ ]:
!apt-get install -y -qq libboost-all-dev

In [ ]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

In [ ]:
!cd LightGBM/python-package/;python3 setup.py install --precompile

In [ ]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

In [ ]:
!nvidia-smi

In [ ]:
param = {
        'num_leaves': 10,
        'max_bin': 127,
        'min_data_in_leaf': 11,
        'learning_rate': 0.02,
        'min_sum_hessian_in_leaf': 0.00245,
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': 0.05,
        'lambda_l1': 4.972,
        'lambda_l2': 2.276,
        'min_gain_to_split': 0.65,
        'max_depth': 14,
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    }

In [ ]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

In [ ]:
%%time
nfold = 2

target = 'target'
predictors = application_train.columns.values.tolist()

skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2019)

oof = np.zeros(len(application_train))
predictions = np.zeros(len(application_test))

i = 1
for train_index, valid_index in skf.split(application_train, y.values):
    print("\nfold {}".format(i))
    xg_train = lgb.Dataset(application_train.iloc[train_index][predictors].values,
                           label=y.iloc[train_index].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(application_train.iloc[valid_index][predictors].values,
                           label=y.iloc[valid_index].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )   

    
    clf = lgb.train(param, xg_train, 5000, valid_sets = [xg_valid], verbose_eval=50, early_stopping_rounds = 50)
    oof[valid_index] = clf.predict(application_train.iloc[valid_index][predictors].values, num_iteration=clf.best_iteration) 
    
    predictions += clf.predict(application_test[predictors], num_iteration=clf.best_iteration) / nfold
    i = i + 1

print("\n\nCV AUC: {:<0.2f}".format(metrics.roc_auc_score(y.values, oof)))

Модель показывает ROC_AUC 0.75 на тренировочных данных

In [ ]:
sample_submission = pd.read_csv(path+'sample_submission.csv')
sample_submission

In [ ]:
my_submission = pd.DataFrame({'SK_ID_CURR': application_test.index, 'TARGET': predictions})
my_submission

In [ ]:
my_submission.to_csv('submission.csv', index=False)

In [ ]:
my_submission

In [ ]:
from IPython.display import FileLink
FileLink('submission.csv')